# Project 2. Yandex Banking

## Project description:

The client is the bank's credit department. We need to find out whether **the client's marital status and number of children affect the repayment of the loan on time**. The input from the bank is statistics on the repayment capacity of the clients.

## Source data and their description: 

* *children* — number of children in the family;
* *days_employed* — total work experience in days;
* *dob_years* — customer age in years;
* *education* — customer education level;
* *education_id* — education level identifier;
* *family_status* — marital status;
* *family_status_id* — marital status identifier;
* *gender* — customer gender;
* *income_type* — type of employment;
* *debt* — was in arrears in repaying loans;
* *total_income* — monthly income;
* *purpose* — loan purpose;

In [1]:
import pandas as pd                      
import numpy as np                       

iPath = 'C:/Users/Churiulin/Desktop/Yandex/Projects'

data = pd.read_csv(f'{iPath}/borrower_reliability_project.csv') 

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Based on the information received, the following preliminary conclusions can be drawn:
1. There are gaps in the received dataframe;
2. There are duplicates in the received dataframe;
3. The data in the column **days_employed** looks strange. It is better to use another data format. There are positive and negative values; 
4. Column **education_id** raises some questions
5. Column **total_income** looks awkward
6. Column **purpose** needs to be reduced to one form. There are phrases with the same gist but written differently - need to put things in order.

## Data pre-processing

### Searching for gross errors

In [3]:
data.columns.to_list()

['children',
 'days_employed',
 'dob_years',
 'education',
 'education_id',
 'family_status',
 'family_status_id',
 'gender',
 'income_type',
 'debt',
 'total_income',
 'purpose']

In the column names, there are no spaces, the names are written in a good style and all characters are lowercase. **BUT**
1. In the text columns the information is written using different registers. Let's get it all in the same style.

In [4]:
# 1. Function `change_registr` - is needed to bring all row columns into the same style
#              input_parameters : data_series - dataframe column
#              output_parameters: ts          - new column 
def change_registr(data_series):
    ts = data_series.str.lower()
    return ts


text_columns = ['education', 'family_status', 
                'gender'   , 'income_type'  ,
                'purpose'  ]              


for column in text_columns:
    data[column] = change_registr(data[column])


data.head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,m,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи


2. Check for errors or strange values in the data and correct them if possible:

We obtain a complete list of columns and use it to find the unique values and number of values for each unique category. The columns **days_employed**, **total_income** and **purpose** will not be considered at this point, as a large number of different options are possible in these columns. 

In [5]:
data.columns # 

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [6]:
column_list = ['children'     , 'dob_years'       , 'education', 'education_id',
               'family_status', 'family_status_id', 'gender'   , 'income_type' ,
               'debt'         ]

for col_name in column_list:
    print(f'Уникальные значения в столбце: {col_name}', data[col_name].unique(), '\n')
    display(data.groupby(col_name)[col_name].count().head(10))

Уникальные значения в столбце: children [ 1  0  3  2 -1  4 20  5] 



children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Уникальные значения в столбце: dob_years [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75] 



dob_years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
Name: dob_years, dtype: int64

Уникальные значения в столбце: education ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень'] 



...

gender
f      14236
m       7288
xna        1
Name: gender, dtype: int64

Уникальные значения в столбце: income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете'] 



income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: income_type, dtype: int64

Уникальные значения в столбце: debt [0 1] 



debt
0    19784
1     1741
Name: debt, dtype: int64

**Conclusions:**

* **children** column - there are strange values, for example there are values: **-1 child** - 47 cases and 20 cases where a person has **20 or more children**.

* **dob_years** column - there are strange values, for example there are values **when a person's return is incorrect**. Based on the results of the check, 101 cases were found where the return value is 0.

* **gender** - strange values are present, for example there is a value: **xna** - which looks strange.

* **education** and **education_id** have no gross errors and agree well with each other.

* **family_status** and **family_status_id** columns do not contain gross errors and agree well with each other.

* **income_type** and **debt** contain no gross errors.


Correct the errors in the **children**, **dob_years** and **gender** columns:

In [7]:
# Column children 
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)

# Column dob_years 
data['dob_years'] = data['dob_years'].replace(0, np.nan)

# Column gender 
data = data.drop(labels = [10701], axis = 0).reset_index(drop = True) 

I do an additional check and see if the errors have been corrected

In [8]:
column_list = ['children', 'dob_years', 'gender']

for col_name in column_list:
    print(f'Unic values in: {col_name}', data[col_name].unique(), '\n')
    display(data.groupby(col_name)[col_name].count().head(10))

Unic values in: children [1 0 3 2 4 5] 



children
0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Unic values in: dob_years [42. 36. 33. 32. 53. 27. 43. 50. 35. 41. 40. 65. 54. 56. 26. 48. 24. 21.
 57. 67. 28. 63. 62. 47. 34. 68. 25. 31. 30. 20. 49. 37. 45. 61. 64. 44.
 52. 46. 23. 38. 39. 51. nan 59. 29. 60. 55. 58. 71. 22. 73. 66. 69. 19.
 72. 70. 74. 75.] 



dob_years
19.0     14
20.0     51
21.0    111
22.0    183
23.0    254
24.0    263
25.0    357
26.0    408
27.0    493
28.0    503
Name: dob_years, dtype: int64

Unic values in: gender ['f' 'm'] 



gender
f    14236
m     7288
Name: gender, dtype: int64

 Часть ошибок удалось исправить. Но теперь в столбце **dob_years** появился nan. Настало время занятся пропусками.

### Missing values

Count the nan values and determine how many total values are missing in our columns. There are gaps in three columns **days_employed**, **total_income** and **dob_years**. 

In [9]:
print(data.isnull().sum()) # get sum

children               0
days_employed       2174
dob_years            101
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


The main reasons for nan values:

* Gaps in the **dob_years** column were created by me in order to correct incorrect return ages. Gaps can easily be corrected based on data from the **income_type** column.

In [10]:
print('Nan values before:', data['dob_years'].isna().sum(), '\n')

for group_age in data['income_type'].unique():
    median_value = data.loc[data['income_type'] == group_age, 'dob_years' ].median()
    data.loc[(data['dob_years'].isna()) & (data['income_type'] == group_age), 'dob_years'] = median_value
    
print('Nan values after:', data['dob_years'].isna().sum(), '\n')

Nan values before: 101 

Nan values after: 0 



* The number of gaps in the **days_employed** and **total_income** columns are the same, suggesting a link between these figures. For example: no job --> no salary, or the client refused to provide this information.

Assume that: 
1. The gaps in **total_income** column should be filled depending on the values in **days_employed**. 
2. The gaps in **days_employed** column should be filled depending on the values in **dob_years**, **education**, **gender**.

In [11]:
data['days_employed'] = abs(data['days_employed'])


print('Nan values before:', data['days_employed'].isna().sum(), '\n')

medians = ((data.groupby(['dob_years',
                         'education', 
                         'gender'   ]))
                .agg({'days_employed':'median'})
                .rename(columns = {'days_employed':'median_days_employed'})
          )

data = data.merge(medians, on = ['dob_years',
                                 'education',
                                 'gender'   ])


data.loc[data['days_employed'].isna(), 'days_employed'] = data.loc[data['days_employed'].isna(), 'median_days_employed']

print('Nan values after:', data['days_employed'].isna().sum(), '\n')


Nan values before: 2174 

Nan values after: 3 



After filling in the blanks, there are 3 lines where the blanks could not be corrected. Let's delete these lines according to their index.

In [12]:
print('before', data.shape)
index_del_rows = data.loc[data['days_employed'].isna()]
print(index_del_rows.index)

# Delete row and reindex
data = data.drop(labels = [21479, 21510, 21516], axis = 0).reset_index(drop = True) 

before (21524, 13)
Int64Index([21479, 21510, 21516], dtype='int64')


Additional steps to make the **days_employed** column look satisfactory:

In [13]:
data['days_employed'] = data['days_employed'].astype('int')

# Sort the values in the days_employed column in descending order and see if all is well:
display(data.sort_values('days_employed', ascending=False).head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed
13238,0,401755,56.0,среднее,1,вдовец / вдова,2,f,пенсионер,0,176278.441171,ремонт жилью,335974.254438
20731,0,401715,69.0,высшее,0,не женат / не замужем,4,f,пенсионер,0,57390.256908,получение образования,370420.179001
6034,1,401675,61.0,среднее,1,женат / замужем,0,f,пенсионер,0,126214.519212,операции с жильем,347226.626654
19064,0,401674,60.0,среднее,1,женат / замужем,0,m,пенсионер,0,325395.724541,автомобили,5635.422839
6038,0,401663,61.0,среднее,1,гражданский брак,1,f,пенсионер,0,48286.441362,свадьба,347226.626654
13210,0,401635,56.0,среднее,1,женат / замужем,0,f,пенсионер,0,48242.322502,покупка недвижимости,335974.254438
3550,0,401619,63.0,среднее,1,гражданский брак,1,f,пенсионер,0,51449.788325,сыграть свадьбу,362963.961475
9961,0,401614,59.0,среднее,1,женат / замужем,0,f,пенсионер,0,152769.694536,покупка жилья для сдачи,353311.040246
13280,0,401591,56.0,среднее,1,в разводе,3,f,пенсионер,0,39513.517543,получение дополнительного образования,335974.254438
13671,0,401590,58.0,среднее,1,женат / замужем,0,f,пенсионер,0,175306.312902,образование,340243.384325


In [14]:
print('before', data.shape)
index_del_rows = data.loc[data['days_employed'].isna()]
print(index_del_rows.index)
print(data.isnull().sum()) # get sum

before (21521, 13)
Int64Index([], dtype='int64')
children                   0
days_employed              0
dob_years                  0
education                  0
education_id               0
family_status              0
family_status_id           0
gender                     0
income_type                0
debt                       0
total_income            2171
purpose                    0
median_days_employed       0
dtype: int64


Fix strange values in the **days_employed** column, which are related to a different type of temporary data storage (most likely using data from another database). New data will be added to the **new_days_employed** column. 

In [15]:
def change_year(row):
    MIN_STAG = 16                   # The minimum age at which you can start working   
    NUM_DAYS = 365                  # Number of days per year, excluding leap years
    
    year = row['dob_years']
    stag = row['days_employed']
    
    if stag > ((year - MIN_STAG) * NUM_DAYS):
        return(stag / 24)
    else:
        return stag
    
data['new_days_employed'] = data.apply(change_year, axis = 1)               
data['new_days_employed'] = data['new_days_employed'].astype('int')

In [16]:
display(data.sort_values('days_employed', ascending=False).head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed,new_days_employed
13238,0,401755,56.0,среднее,1,вдовец / вдова,2,f,пенсионер,0,176278.441171,ремонт жилью,335974.254438,16739
20731,0,401715,69.0,высшее,0,не женат / не замужем,4,f,пенсионер,0,57390.256908,получение образования,370420.179001,16738
6034,1,401675,61.0,среднее,1,женат / замужем,0,f,пенсионер,0,126214.519212,операции с жильем,347226.626654,16736
19064,0,401674,60.0,среднее,1,женат / замужем,0,m,пенсионер,0,325395.724541,автомобили,5635.422839,16736
6038,0,401663,61.0,среднее,1,гражданский брак,1,f,пенсионер,0,48286.441362,свадьба,347226.626654,16735
13210,0,401635,56.0,среднее,1,женат / замужем,0,f,пенсионер,0,48242.322502,покупка недвижимости,335974.254438,16734
3550,0,401619,63.0,среднее,1,гражданский брак,1,f,пенсионер,0,51449.788325,сыграть свадьбу,362963.961475,16734
9961,0,401614,59.0,среднее,1,женат / замужем,0,f,пенсионер,0,152769.694536,покупка жилья для сдачи,353311.040246,16733
13280,0,401591,56.0,среднее,1,в разводе,3,f,пенсионер,0,39513.517543,получение дополнительного образования,335974.254438,16732
13671,0,401590,58.0,среднее,1,женат / замужем,0,f,пенсионер,0,175306.312902,образование,340243.384325,16732


Correct the omissions in the **total_income** column depending on **new_days_employed** and **education**. 

Unfortunately from the many different options in **new_days_employed** it is not possible to fill all gaps in **total_income** at once using only the previously used method. A large number of gaps remain. That's why let's create an additional column **days_employed_kateg** to avoid losing data. This column contains textual information about the employment history broken down into categories to simplify the task for Python.

In [17]:
def stag_by_category(row):
    ONE_YEAR = 365
    stag = row['new_days_employed']
    
    if 0 < stag < (ONE_YEAR * 5):
        return 'Seniority 1 - 5 years'
    elif (ONE_YEAR * 5) < stag < (ONE_YEAR * 10):
        return 'Seniority 5 - 10 years'
    elif (ONE_YEAR * 10) < stag < (ONE_YEAR * 15):
        return 'Seniority 10 - 15 years'
    elif (ONE_YEAR * 15) < stag < (ONE_YEAR * 20):
        return 'Seniority 15 - 20 years'
    elif (ONE_YEAR * 20) < stag < (ONE_YEAR * 25):
        return 'Seniority 20 - 25 years'
    elif (ONE_YEAR * 25) < stag < (ONE_YEAR * 30):
        return 'Seniority 25 - 30 years'
    else:
        return 'more 30 years'
    
data['days_employed_kateg'] = data.apply(stag_by_category, axis = 1)

Finally, can remove all omissions from the **total_income** column 

In [18]:
medians = ((data.groupby(['days_employed_kateg',
                        'education']))
                .agg({'total_income':'median'})
                .rename(columns = {'total_income':'median_total_income'})
          )

data = data.merge(medians, on = ['days_employed_kateg',
                                 'education'])

data.loc[data['total_income'].isna(), 'total_income'] = data.loc[data['total_income'].isna(), 'median_total_income']

data['total_income'] = data['total_income'].astype('int')

Checking again for gaps

In [19]:
print(data.isnull().sum())

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
median_days_employed    0
new_days_employed       0
days_employed_kateg     0
median_total_income     0
dtype: int64


Great, there are no more nan values, we can start the next step.

### Duplicates

*Task to find duplicates and remove them*. **The main reasons** for duplicates may be several:
+ When merging different databases, the same person may have been added more than once.
+ At some point, when the person filled in the relevant data, the computer hangs after pressing the submit button and then generates an error. The data was sent to the database, but the person was unaware of this and re-entered the data.

In [20]:
print(data.duplicated().sum()) # Counting obvious duplicates in the table

71


In [21]:
data = data.drop_duplicates().reset_index(drop = True)  # removing obvious duplicates
                                                        # (with deletion of old indexes and formation of new ones)

In [22]:
print(data.duplicated().sum())

0


A search for implicit duplicates, or incorrect names, has been performed previously, so this search will not be repeated in this section.

### Lemmatisation

In [23]:
from pymystem3 import Mystem        # 
from collections import Counter

m = Mystem() 

lemma = m.lemmatize(' '.join(data['purpose']))
print(Counter(lemma)) 

Counter({' ': 55012, 'недвижимость': 6348, 'покупка': 5896, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2917, 'операция': 2602, 'свадьба': 2323, 'свой': 2229, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 837, 'проведение': 768, 'сыграть': 764, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 626, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 127, '\n': 1})


On the basis of lemmatisation, we determined that there are some of the most frequent words that we can use to create special categories (and it is better to use nouns rather than verbs):\

1. real estate and housing;
2. car;
3. education;
4. wedding

In [24]:
def purpose_by_category(row):
    purpose = row['purpose']
    
    if ('жиль' in purpose) or ('недвиж' in purpose):
        return 'property'
    elif ('авто' in purpose):
        return 'авто'
    elif ('образов' in purpose):
        return 'education'
    else:
        return 'wedding'
 
data['new_purpose'] = data.apply(purpose_by_category, axis = 1)

data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed,new_days_employed,days_employed_kateg,median_total_income,new_purpose
0,1,8437,42.0,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,2608.881632,8437,Seniority 20 - 25 years,175888.29196,property
1,1,8899,42.0,высшее,0,гражданский брак,1,f,сотрудник,0,360887,сыграть свадьбу,2608.881632,8899,Seniority 20 - 25 years,175888.29196,wedding
2,0,8963,42.0,высшее,0,женат / замужем,0,f,сотрудник,0,164480,покупка недвижимости,2608.881632,8963,Seniority 20 - 25 years,175888.29196,property
3,0,8157,42.0,высшее,0,вдовец / вдова,2,f,госслужащий,0,307192,операции со своей недвижимостью,2608.881632,8157,Seniority 20 - 25 years,175888.29196,property
4,1,7587,42.0,высшее,0,не женат / не замужем,4,f,сотрудник,0,113663,строительство недвижимости,2608.881632,7587,Seniority 20 - 25 years,175888.29196,property


There is one last column which needs to be turned around before answering the questions. Namely the column with the salaries. Let's create categories:

In [25]:
def salary_by_category(row):
    salary = row['total_income']
    
    if 0 < salary <= 25000:
        return 'very poor'
    elif 25000 < salary <= 50000:
        return 'poor'
    elif 50000 < salary <= 100000:
        return 'middle class'
    elif 100000 < salary <= 200000:
        return 'almost rich'
    else:
        return 'rich'
    
data['new_salary'] = data.apply(salary_by_category, axis = 1)
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed,new_days_employed,days_employed_kateg,median_total_income,new_purpose,new_salary
0,1,8437,42.0,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,2608.881632,8437,Seniority 20 - 25 years,175888.29196,property,rich
1,1,8899,42.0,высшее,0,гражданский брак,1,f,сотрудник,0,360887,сыграть свадьбу,2608.881632,8899,Seniority 20 - 25 years,175888.29196,wedding,rich
2,0,8963,42.0,высшее,0,женат / замужем,0,f,сотрудник,0,164480,покупка недвижимости,2608.881632,8963,Seniority 20 - 25 years,175888.29196,property,almost rich
3,0,8157,42.0,высшее,0,вдовец / вдова,2,f,госслужащий,0,307192,операции со своей недвижимостью,2608.881632,8157,Seniority 20 - 25 years,175888.29196,property,rich
4,1,7587,42.0,высшее,0,не женат / не замужем,4,f,сотрудник,0,113663,строительство недвижимости,2608.881632,7587,Seniority 20 - 25 years,175888.29196,property,almost rich


The preprocessing of the data resulted in three additional columns:**days_employed_kateg**, **new_purpose** and **new_salary**, proceeding to answer the questions.

## Answer to the questions

### Is there a correlation between having children and paying back the loan on time?

In [26]:
report = data.groupby('children').agg({'debt':['mean', 'count']})

report = report.rename(columns = {'debt' :'Descriptive statistics'}, level = 0)
report = report.rename(columns = {'count':'Number of borrowers'   ,
                                  'mean' :'%non-refundable'       }, level = 1)

report.index = report.index.rename('Number of children')

report.style.format(formatter={('Descriptive statistics', '%non-refundable'    ): "{:.2%}",
                               ('Descriptive statistics', 'Number of borrowers'): "{:.1f}"})

**Conclusion**: Childless people have less debt.

### Is there a correlation between marital status and repayment on time?

In [27]:
report = data.groupby('family_status').agg({'debt':['mean', 'count']})

report = report.rename(columns = {'debt' :'Descriptive statistics'}, level = 0)
report = report.rename(columns = {'count':'Number of borrowers',
                                  'mean' :'%non-refundable'       }, level = 1)

report.index = report.index.rename('Marital status')

report.style.format(formatter={('Descriptive statistics', '%non-refundable'    ): "{:.2%}",
                               ('Descriptive statistics', 'Number of borrowers'): "{:.1f}"})

**Conclusion**: A strict correlation is still problematic. The data show that there are two categories of citizens (common-law and unmarried) where the default rate is higher, but a difference of 3% does not allow a confident answer to the question that married couples always repay on time.

### Is there a correlation between income and repayment on time?

In [28]:
# Group data
report = data.groupby('new_salary').agg({'debt':['mean', 'count']})

# Rename columns 
report = report.rename(columns = {'debt' :'Descriptive statistics'}, level = 0)
report = report.rename(columns = {'count':'Number of borrowers',
                                  'mean' :'%non-refundable'       }, level = 1)

# Rename index
report.index = report.index.rename('Income level')

# Change output format --> only view
report.style.format(formatter={('Descriptive statistics', '%non-refundable'    ): "{:.2%}",
                               ('Descriptive statistics', 'Number of borrowers'): "{:.1f}"})

**Conclusion**: The data show that there is a correlation between the percentage of defaults and the level of wages. However, people with low income try to pay back their loans on time. 

### How do the different purposes of a loan affect its repayment on time?

In [29]:
report = data.groupby('new_purpose').agg({'debt':['mean', 'count']})

report = report.rename(columns = {'debt' :'Descriptive statistics'}, level = 0)
report = report.rename(columns = {'count':'Number of borrowers',
                                  'mean' :'%non-refundable'       }, level = 1)

report.index = report.index.rename('Purpose of the loan')

report.style.format(formatter={('Descriptive statistics', '%non-refundable'    ): "{:.2%}",
                               ('Descriptive statistics', 'Number of borrowers'): "{:.1f}"})

**Conclusion**: The highest percentage of defaults on a loan on time is observed when taking out a loan for educational purposes. This fact can be explained by the fact that education costs money and the benefits of education may not manifest themselves immediately. The largest number of applications is for real estate, with people trying to pay back such loans on time and the default rate for this category is the lowest. This fact can also be explained by the fact that people decide to buy real estate with a stable job and salary (in most cases), which allows them to make loan payments on time.

## General conclusion

In conclusion, it can be noted that all of the above findings are controversial and further statistical analysis of the data is required to unequivocally answer the questions posed. Grouping alone is clearly insufficient. However, it can be noted that people who take out a loan in order to buy real estate are the most reliable. This is primarily due to the fact that married people with children are more likely to need to buy their own home. In addition, these people usually have stable jobs and a certain level of education, which can guarantee them some confidence. For this reason, home loans have the lowest default rate.

The highest risk of default may be observed in unmarried people without children who take out a loan for education or to buy a car. This fact can also be easily explained by the human factor. As a rule - these are young people who don't have stable income and that doesn't allow them to make payments on time. Such loans are more risky. 